<a href="https://colab.research.google.com/github/huytuan3108/btap_AI/blob/main/ID_card.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Trích xuất thông tin từ thẻ CCCD và lưu vào Excel
# Chạy trên Google Colab

# Bước 1: Cài đặt các thư viện cần thiết
!pip install easyocr
!pip install opencv-python
!pip install pandas openpyxl

# Bước 2: Import các thư viện
import cv2
import easyocr
import pandas as pd
from google.colab import files
import re
import numpy as np

# Bước 3: Tải ảnh CCCD lên Colab
print("Vui lòng upload ảnh CCCD (định dạng JPG hoặc PNG):")
uploaded = files.upload()

# Lấy tên file ảnh
image_path = list(uploaded.keys())[0]
print(f"Đã upload ảnh: {image_path}")

# Bước 4: Đọc và tiền xử lý ảnh
# Đọc ảnh bằng OpenCV
image = cv2.imread(image_path)

# Chuyển ảnh sang grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Tăng độ tương phản
gray = cv2.convertScaleAbs(gray, alpha=1.5, beta=0)

# Làm mờ để giảm nhiễu
gray = cv2.GaussianBlur(gray, (5, 5), 0)

# Bước 5: Trích xuất văn bản bằng EasyOCR
# Khởi tạo EasyOCR với ngôn ngữ tiếng Việt và tiếng Anh
reader = easyocr.Reader(['vi', 'en'], gpu=False)

# Trích xuất văn bản
result = reader.readtext(gray, paragraph=True)
text = " ".join([res[1] for res in result])

# In văn bản trích xuất để kiểm tra
print("\nVăn bản trích xuất từ ảnh:")
print(text)

# Bước 6: Phân tích thông tin cá nhân
def extract_info(text):
    info = {}

    # Số CCCD
    id_pattern = r'\b\d{9}\b|\b\d{12}\b'
    id_match = re.search(id_pattern, text)
    info['Số CCCD'] = id_match.group() if id_match else 'Không tìm thấy'

    # Họ và tên
    name_pattern = r'(Họ và tên|Full name)\s*[:\-\s]*(.*?)(?:\n|$|\d)'
    name_match = re.search(name_pattern, text, re.IGNORECASE)
    info['Họ và tên'] = name_match.group(2).strip() if name_match else 'Không tìm thấy'

    # Ngày sinh
    dob_pattern = r'(Ngày sinh|Date of birth)\s*[:\-\s]*(\d{1,2}[-/\s]\d{1,2}[-/\s]\d{4})'
    dob_match = re.search(dob_pattern, text, re.IGNORECASE)
    info['Ngày sinh'] = dob_match.group(2).strip() if dob_match else 'Không tìm thấy'

    # Giới tính
    sex_pattern = r'(Giới tính|Sex)\s*[:\-\s]*(Nam|Nữ|Male|Female)'
    sex_match = re.search(sex_pattern, text, re.IGNORECASE)
    info['Giới tính'] = sex_match.group(2).strip() if sex_match else 'Không tìm thấy'

    # Quê quán
    hometown_pattern = r'(Quê quán|Place of origin)\s*[:\-\s]*(.*?)(?:Nơi thường trú|Place of residence|\d)'
    hometown_match = re.search(hometown_pattern, text, re.IGNORECASE)
    info['Quê quán'] = hometown_match.group(2).strip() if hometown_match else 'Không tìm thấy'

    # Nơi thường trú
    address_pattern = r'(Nơi thường trú|Place of residence)\s*[:\-\s]*(.*?)(?:\n|$)'
    address_match = re.search(address_pattern, text, re.IGNORECASE)
    info['Nơi thường trú'] = address_match.group(2).strip() if address_match else 'Không tìm thấy'

    return info

# Trích xuất thông tin
extracted_info = extract_info(text)
print("\nThông tin trích xuất:")
for key, value in extracted_info.items():
    print(f"{key}: {value}")

# Bước 7: Lưu thông tin vào file Excel
# Tạo DataFrame từ thông tin trích xuất
df = pd.DataFrame([extracted_info])

# Lưu vào file Excel
output_file = 'cccd_info.xlsx'
df.to_excel(output_file, index=False)
print(f"\nĐã lưu thông tin vào file: {output_file}")

# Tải file Excel về máy
files.download(output_file)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

Saving cccd.jpg to cccd.jpg
Đã upload ảnh: cccd.jpg
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete
Văn bản trích xuất từ ảnh:
CĂN CƯỚC CÔNG DÂN Citizen Identity Card Số / No:: 075206001761 Họ và tên / Full name: ĐĂNG HUY TUẨN Ngày sinh / Date of birth: 31/08/2006 Giới tính / Sex: Nam Quôc tich / Nationality:   Việt Nam Quê quán / Place pf origin: Phương Tú; Ứng Hòa, Hà Nội Nơi thường trú / Place of residence 548, Tổ 4, Kp 12 An Bình, Thành phố Biên Hòa, Đông Áai

Thông tin trích xuất:
Số CCCD: 075206001761
Họ và tên: / Full name: ĐĂNG HUY TUẨN Ngày sinh / Date of birth:
Ngày sinh: 31/08/2006
Giới tính: Nam
Quê quán: / Place pf origin: Phương Tú; Ứng Hòa, Hà Nội
Nơi thường trú: / Place of residence 548, Tổ 4, Kp 12 An Bình, Thành phố Biên Hòa, Đông Áai

Đã lưu thông tin vào file: cccd_info.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>